In [2]:
import re

import pandas as pd
import numpy as np
from tqdm import tqdm

import keras
from keras import losses, metrics, layers, ops

In [3]:
train_df = pd.read_csv('data/train.csv')

In [4]:
train_set_df = train_df[['agent1', 'agent2', 'EnglishRules', 'LudRules']]

In [26]:
lud_preprocessor = layers.TextVectorization(
    vocabulary='ludii_tokens.dic',
    standardize='strip_punctuation'
)

eng_preprocessor = layers.TextVectorization(
    vocabulary='eng_tokens.dic'
)

max_eng = 0
max_lud = 0

eng_uniques = train_set_df['EnglishRules'].unique()

for engrul in tqdm(eng_uniques):
    eng_vector = eng_preprocessor(engrul)
    max_eng = max(max_eng, eng_vector.shape[0])

print(max_eng)


100%|██████████| 1328/1328 [00:06<00:00, 215.91it/s]

858


In [23]:
lud_uniques = train_set_df['LudRules'].unique()

EQPMNT_RE = r'\(equipment'
RULESTRIP_RE = r'[^a-zA-Z\(\)\{\}]'

for rule in lud_uniques:
    start = re.search(EQPMNT_RE, rule).span()[0]
    pure_rule = rule[start:]
    pure_rule = re.sub(RULESTRIP_RE, ' ', pure_rule)
    lud_vector = lud_preprocessor(pure_rule)
    max_lud = max(max_lud, lud_vector.shape[0])

print(max_lud)

23613


In [30]:
agent1_uniques = train_set_df['agent1'].unique()
agent2_uniques = train_set_df['agent2'].unique()

set(agent1_uniques) >= set(agent2_uniques)

code_agent = {}
agent_code = {}

for id, agent in enumerate(agent1_uniques):
    code_agent[id] = agent
    agent_code[agent] = id


{0: 'MCTS-ProgressiveHistory-0.1-MAST-false', 1: 'MCTS-ProgressiveHistory-0.1-MAST-true', 2: 'MCTS-ProgressiveHistory-0.1-NST-false', 3: 'MCTS-ProgressiveHistory-0.1-NST-true', 4: 'MCTS-ProgressiveHistory-0.1-Random200-false', 5: 'MCTS-ProgressiveHistory-0.1-Random200-true', 6: 'MCTS-ProgressiveHistory-0.6-MAST-false', 7: 'MCTS-ProgressiveHistory-0.6-MAST-true', 8: 'MCTS-ProgressiveHistory-0.6-NST-false', 9: 'MCTS-ProgressiveHistory-0.6-NST-true', 10: 'MCTS-ProgressiveHistory-0.6-Random200-false', 11: 'MCTS-ProgressiveHistory-0.6-Random200-true', 12: 'MCTS-ProgressiveHistory-1.41421356237-MAST-false', 13: 'MCTS-ProgressiveHistory-1.41421356237-MAST-true', 14: 'MCTS-ProgressiveHistory-1.41421356237-NST-false', 15: 'MCTS-ProgressiveHistory-1.41421356237-NST-true', 16: 'MCTS-ProgressiveHistory-1.41421356237-Random200-false', 17: 'MCTS-ProgressiveHistory-1.41421356237-Random200-true', 18: 'MCTS-UCB1-0.1-MAST-false', 19: 'MCTS-UCB1-0.1-MAST-true', 20: 'MCTS-UCB1-0.1-NST-false', 21: 'MCTS-UC

eng_vector.shape=(900,)

lud_vector.shape=(23700,)

agents_to_categorical.shape=(144)

In [31]:
agents_len = 144
engvector_len = 900
ludvector_len = 23700

In [43]:
# 900+23700+144 = 24744
# defining input shape and input data distribution

enter = layers.Input(shape=(24744,))
agent_input = ops.slice(enter, (0,), (agents_len,))
engrul_input = ops.slice(enter, (agents_len,), (engvector_len,))
ludrul_input = ops.slice(enter, (agents_len+engvector_len,), (ludvector_len,))